# Update net

In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
from main import CIFAR10_dataset, CIFAR10_trainer
import torch
import torchvision
import torch.nn as nn
from tqdm import tqdm
import multiprocessing
import torch.optim as optim
import torch.nn.functional as  F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.data import default_collate
from torchvision.transforms import v2

In [25]:
import torchvision.transforms as transforms

def add_gaussian_noise(img, mean=0, std=0.1):
    noise = torch.randn(img.size()) * std + mean
    return img + noise

# Transformaciones para entrenamiento
train_transforms = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomRotation(degrees=15),
    # transforms.RandomGrayscale(p=0.1),
    # transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3)),
    transforms.Lambda(lambda x: add_gaussian_noise(x, 0, 0.1)),
])


train_dataset = CIFAR10_dataset(partition="train", transform=train_transforms)
test_dataset = CIFAR10_dataset(partition="test")


####################################################################
# DataLoader Class
####################################################################

batch_size = 150
num_workers = multiprocessing.cpu_count()-1
print("Num workers", num_workers)
train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=num_workers)
test_dataloader = DataLoader(test_dataset, batch_size, shuffle=False, num_workers=num_workers)


Loading CIFAR10  train  Dataset...
Files already downloaded and verified
	Total Len.:  50000 
 --------------------------------------------------

Loading CIFAR10  test  Dataset...
Files already downloaded and verified
	Total Len.:  10000 
 --------------------------------------------------
Num workers 11


In [34]:


####################################################################
# Neural Network Class
####################################################################

# Define the CNN model
# Define ResBlocks to create a CNN

class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResBlock, self).__init__()
        
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv_shortcut = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride)
        self.relu = nn.ReLU()

    def forward(self, x):
        shortcut = self.conv_shortcut(x)

        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += shortcut
        out = self.relu(out)

        return out


# Define the CNN model
class CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CNN, self).__init__()
        layers=[ResBlock(3, 32, stride=1)]
        channels = [32, 64, 128, 256]
        for  c in channels:

            layers.append(ResBlock(c, c , stride=1))

            layers.append(ResBlock(c , c * 2, stride=2))
        layers.append(ResBlock(512, 512, stride=1))
        self.conv_layers = nn.Sequential(
            *layers
            )
        
        with torch.no_grad():
            input = torch.zeros(1, 3, 32, 32)
            output = self.conv_layers(input)
            size = output.view(1, -1).shape[1]

        
        self.fc1 = nn.Linear(size, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):

        x = self.conv_layers(x)
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x


In [35]:
# Instantiating the network and printing its architecture
num_classes = 10
net = CNN(num_classes)
print(net)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Params: ", count_parameters(net))

####################################################################
# Training settings
####################################################################

# Training hyperparameters
epochs = 200
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters(), lr=0.002, weight_decay=1e-4)
lr_scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=0.01, steps_per_epoch=len(train_dataloader), epochs=epochs
)

CNN(
  (conv_layers): Sequential(
    (0): ResBlock(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv_shortcut): Conv2d(3, 32, kernel_size=(1, 1), stride=(1, 1))
      (relu): ReLU()
    )
    (1): ResBlock(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv_shortcut): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
      (relu): ReLU()
    )
    (2): ResBlock(
      (conv): Conv2

In [36]:
trainer = CIFAR10_trainer(net, train_dataloader, test_dataloader, optimizer,criterion, epochs, lr_scheduler)

In [37]:
trainer.train()


---- Start Training ----


Test 0: 100%|██████████| 67/67 [00:25<00:00,  2.68batch/s]


[Epoch 1] Train Loss: 1.100268 - Test Loss: 0.934889 - Train Error: 62.24% - Test Error: 50.25%


Test 1: 100%|██████████| 67/67 [00:25<00:00,  2.67batch/s]


[Epoch 2] Train Loss: 0.777286 - Test Loss: 0.719255 - Train Error: 41.63% - Test Error: 38.51%


...

[Epoch 22] Train Loss: 0.033527 - Test Loss: 0.620853 - Train Error: 1.66% - Test Error: 18.86%



Test 22: 100%|██████████| 67/67 [00:23<00:00,  2.83batch/s]

[Epoch 23] Train Loss: 0.033198 - Test Loss: 0.653464 - Train Error: 1.63% - Test Error: 19.56%



Epoch 23:  75%|███████▌  | 251/334 [8:14:07<2:43:23, 118.12s/batch]   


KeyboardInterrupt: 